# LangChain 核心模块学习：Model I/O

`Model I/O` 是 LangChain 为开发者提供的一套面向 LLM 的标准化模型接口，包括模型输入（Prompts）、模型输出（Output Parsers）和模型本身（Models）。

- Prompts：模板化、动态选择和管理模型输入
- Models：以通用接口调用语言模型
- Output Parser：从模型输出中提取信息，并规范化内容

![](../images/model_io.jpeg)


In [ ]:
# 去掉pip install 使用项目依赖版本
# ! pip install -U langchain
# pip install -U langchain-openai
# pip install -U jinjia2
# pip install chromadb

## 模型输入 Prompts

一个语言模型的提示是用户提供的一组指令或输入，用于引导模型的响应，帮助它理解上下文并生成相关和连贯的基于语言的输出，例如回答问题、完成句子或进行对话。


- 提示模板（Prompt Templates）：参数化的模型输入
- 示例选择器（Example Selectors）：动态选择要包含在提示中的示例


## 提示模板 Prompt Templates

**Prompt Templates 提供了一种预定义、动态注入、模型无关和参数化的提示词生成方式，以便在不同的语言模型之间重用模板。**

一个模板可能包括指令、少量示例以及适用于特定任务的具体背景和问题。

通常，提示要么是一个字符串（LLMs），要么是一组聊天消息（Chat Model）。


类继承关系:

```
BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate

PromptValue --> StringPromptValue
                ChatPromptValue
```


**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/prompts**



### 使用 PromptTemplate 类生成提升词

**通常，`PromptTemplate` 类的实例，使用Python的`str.format`语法生成模板化提示；也可以使用其他模板语法（例如jinja2）。**

#### 使用 from_template 方法实例化 PromptTemplate

In [1]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# 使用 format 生成提示
prompt = prompt_template.format(adjective="funny", content="chickens")
print(prompt)

Tell me a funny joke about chickens.


In [2]:
print(prompt_template)

input_variables=['adjective', 'content'] input_types={} partial_variables={} template='Tell me a {adjective} joke about {content}.'


In [3]:
prompt_template = PromptTemplate.from_template(
    "Tell me a joke"
)
# 生成提示
prompt = prompt_template.format()
print(prompt)

Tell me a joke


In [4]:
print(prompt_template)

input_variables=[] input_types={} partial_variables={} template='Tell me a joke'


#### 使用构造函数（Initializer）实例化 PromptTemplate

使用构造函数实例化 `prompt_template` 时必须传入参数：`input_variables` 和 `template`。

在生成提示过程中，会检查输入变量与模板字符串中的变量是否匹配，如果不匹配，则会引发异常；

In [8]:
invalid_prompt = PromptTemplate(
    input_variables=["adjective"],
    template="Tell me a {adjective} joke about {content}."
)

传入 content 后才能生成可用的 prompt

In [6]:
valid_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}."
)

In [7]:
print(valid_prompt)

input_variables=['adjective', 'content'] input_types={} partial_variables={} template='Tell me a {adjective} joke about {content}.'


In [11]:
valid_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [12]:
prompt_template = PromptTemplate.from_template(
    "讲{num}个给程序员听得笑话"
)

In [13]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", max_tokens=1000)

prompt = prompt_template.format(num=2)
print(f"prompt: {prompt}")

result = llm.invoke(prompt)
print(f"result: {result}")

prompt: 讲2个给程序员听得笑话
result: ，可怜可怜他们吧

1. 为什么程序员不喜欢使用Windows？因为Windows总是在开发人员最需要的时候崩溃。

2. 一个程序员走进一家餐馆，点了一杯水。服务员问道：“你需要冰块吗？”程序员回答：“不，我已经有一个很大的Bug了，不需要再冰了。”


In [14]:
# 【新增】ChatOpenAI调用方式，使用gpt-4o-mini模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", max_tokens=1000)

prompt = prompt_template.format(num=2)
print(f"prompt: {prompt}")

result = llm.invoke(prompt)
print(f"result: {result}")

prompt: 讲2个给程序员听得笑话
result: content='当然可以！这里有两个适合程序员的笑话：\n\n1. **变量的秘密**\n   - 为什么程序员不喜欢在大海里游泳？  \n   - 因为他们怕有太多的变量（waves）！\n\n2. **调试的尴尬**\n   - 程序员的妻子对他说：“你能帮我把这个灯泡换掉吗？”  \n   - 程序员回答：“我已经调试过了，问题出在代码上，不是灯泡。”\n\n希望你喜欢这两个笑话！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 122, 'prompt_tokens': 17, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEZ9k0CdfBuZPn2zqg3vIb09NE6Pv', 'finish_reason': 'stop', 'logprobs': None} id='run-9acb0ff2-a089-47c2-b9e6-a10ec53292aa-0' usage_metadata={'input_tokens': 17, 'output_tokens': 122, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [15]:
print(llm.invoke(prompt_template.format(num=3)))

content="当然可以！以下是三个适合程序员的笑话：\n\n1. **为什么程序员总是混淆圣诞节和万圣节？**\n   因为Oct 31 = Dec 25。\n\n2. **一个程序员在酒吧里......**\n   酒保问：“你要喝点什么？”\n   程序员说：“我想要一杯 '在我代码里没有错误'。”\n   酒保回答：“这可不是酒！你要不要一杯 '在你代码里调试过的'？”\n\n3. **程序员的魔法：**\n   程序员走进一家餐厅，点了个三明治。服务员问他：“你要什么配料？”\n   程序员回答：“随便，你可以给我加什么就加什么，只要别让我点这个‘字符’。”\n\n希望这些笑话能让你会心一笑！" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 197, 'prompt_tokens': 17, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEZA9QXyyUL8V9b1sSRdN0Uq8w52t', 'finish_reason': 'stop', 'logprobs': None} id='run-d41371ba-80fd-4dd6-8fda-b4cd9b011d96-0' usage_metadata={'input_tokens': 17, 'output_tokens': 197, 'total_tokens': 214, 'input_token_details': {'audio': 0, 'cache_read': 

#### 使用 jinja2 生成模板化提示

In [17]:
jinja2_template = "Tell me a {{ adjective }} joke about {{ content }}"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens'

In [18]:
print(prompt)

input_variables=['adjective', 'content'] input_types={} partial_variables={} template='Tell me a {{ adjective }} joke about {{ content }}' template_format='jinja2'


#### 实测：生成多种编程语言版本的快速排序

In [19]:
sort_prompt_template = PromptTemplate.from_template(
    "生成可执行的快速排序 {programming_language} 代码"
)

In [20]:
print(llm.invoke(sort_prompt_template.format(programming_language="python")))

content='当然可以！下面是一个可执行的快速排序（Quick Sort）Python 代码示例：\n\n```python\ndef quick_sort(arr):\n    if len(arr) <= 1:\n        return arr\n    pivot = arr[len(arr) // 2]  # 选择中间元素作为基准\n    left = [x for x in arr if x < pivot]  # 小于基准的元素\n    middle = [x for x in arr if x == pivot]  # 等于基准的元素\n    right = [x for x in arr if x > pivot]  # 大于基准的元素\n    return quick_sort(left) + middle + quick_sort(right)  # 递归排序并合并结果\n\n# 测试代码\nif __name__ == "__main__":\n    arr = [3, 6, 8, 10, 1, 2, 1]\n    sorted_arr = quick_sort(arr)\n    print("排序后的数组:", sorted_arr)\n```\n\n你可以将这段代码复制到 Python 环境中执行，程序将对给定的数组进行排序并打印出排序后的结果。这个实现使用了递归和列表推导式来简化代码。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 262, 'prompt_tokens': 16, 'total_tokens': 278, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-min

In [21]:
print(llm.invoke(sort_prompt_template.format(programming_language="java")))

content='当然，以下是一个可执行的快速排序（Quick Sort）算法的 Java 实现示例：\n\n```java\npublic class QuickSort {\n\n    // 主方法\n    public static void main(String[] args) {\n        int[] array = {38, 27, 43, 3, 9, 82, 10};  // 示例数组\n        System.out.println("未排序数组: ");\n        printArray(array);\n        \n        quickSort(array, 0, array.length - 1);\n        \n        System.out.println("排序后数组: ");\n        printArray(array);\n    }\n\n    // 快速排序方法\n    public static void quickSort(int[] array, int low, int high) {\n        if (low < high) {\n            // 查找分区点\n            int pivotIndex = partition(array, low, high);\n            // 递归排序分区\n            quickSort(array, low, pivotIndex - 1);\n            quickSort(array, pivotIndex + 1, high);\n        }\n    }\n\n    // 用于分区的辅助方法\n    private static int partition(int[] array, int low, int high) {\n        int pivot = array[high];  // 选择最后一个元素作为基准\n        int i = low - 1; // 小于基准元素的最后一个索引\n\n        for (int j = low; j < high; j++) {\n            

In [22]:
print(llm.invoke(sort_prompt_template.format(programming_language="C++")))

content='下面是一个可执行的快速排序（QuickSort）C++ 示例代码：\n\n```cpp\n#include <iostream>\n#include <vector>\n\nusing namespace std;\n\n// 快速排序算法\nint partition(vector<int>& arr, int low, int high) {\n    int pivot = arr[high]; // 选择最后一个元素作为主元\n    int i = low - 1; // 指向小于主元的元素的索引\n\n    for (int j = low; j <= high - 1; j++) {\n        if (arr[j] < pivot) {\n            i++;\n            swap(arr[i], arr[j]); // 交换当前元素和小于主元的元素\n        }\n    }\n    swap(arr[i + 1], arr[high]); // 将主元放到正确的位置\n    return i + 1; // 返回主元的索引\n}\n\nvoid quickSort(vector<int>& arr, int low, int high) {\n    if (low < high) {\n        int pi = partition(arr, low, high); // 划分数组并返回主元的位置\n\n        // 递归排序主元左边和右边的子数组\n        quickSort(arr, low, pi - 1);\n        quickSort(arr, pi + 1, high);\n    }\n}\n\nint main() {\n    vector<int> arr = {10, 7, 8, 9, 1, 5}; // 测试数组\n    int n = arr.size();\n\n    cout << "原始数组: ";\n    for (int num : arr) cout << num << " ";\n    cout << endl;\n\n    quickSort(arr, 0, n - 1); // 排序\n\n    

## 使用 ChatPromptTemplate 类生成适用于聊天模型的聊天记录

**`ChatPromptTemplate` 类的实例，使用`format_messages`方法生成适用于聊天模型的提示。**

### 使用 from_messages 方法实例化 ChatPromptTemplate

In [23]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

# 生成提示
messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

In [24]:
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


In [25]:
print(messages[0].content)
print(messages[-1].content)

You are a helpful AI bot. Your name is Bob.
What is your name?


In [26]:
from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=1000)

In [27]:
chat_model.invoke(messages)

AIMessage(content='My name is Bob. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 50, 'total_tokens': 63, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEd7naqB1vB6ICxYw3L4QeUpF56GQ', 'finish_reason': 'stop', 'logprobs': None}, id='run-9484d100-c10e-4c77-820a-088925e70af1-0', usage_metadata={'input_tokens': 50, 'output_tokens': 13, 'total_tokens': 63, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 摘要总结

In [28]:
summary_template = ChatPromptTemplate.from_messages([
    ("system", "你将获得关于同一主题的{num}篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。"),
    ("human", "{user_input}"),
])

In [29]:
messages = summary_template.format_messages(
    num=3,
    user_input='''1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。
'''
)

In [30]:
print(messages[-1].content)

1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。



In [31]:
chat_result = chat_model.invoke(messages)

In [32]:
print(chat_result.content)

第一篇文章的论点是PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。写PHP就像做披萨一样简单，只需要把配料扔进一个碗，然后放到服务器上。

第二篇文章的论点是Python是世界上最好的拜金主义者语言，强调了Python的美感、简洁性和空格的重要性，还提到了Python能让人在两行代码中感受到飘逸的花香和清新的微风。

第三篇文章的论点是Java是世界上最好的德育课编程语言，坚守严谨、安全的编程原则，类比Java为一个严格的老师，不怀柔、不让偷懒，教会规范和自律。

在我看来，第三篇文章提出了更好的论点。尽管每篇文章都夸大了所支持程序语言的特点，但第三篇更重点强调了编程的严谨性和规范性，提倡学习者养成良好的编码习惯和自律性。这种态度对于长期提升编程水平和开发质量是至关重要的。虽然编程语言的选择因人而异，但教导人们编写高质量、安全的代码的观点是更具建设性和实用性的。


In [33]:
messages = summary_template.format_messages(
    num=2,
    user_input='''1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。
-----------
2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。
--------
3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，则指的是平常人所讲之道、常俗之道。因此，“道可道，非常道”，指“道”是可以言说的，但它不是平常人所谓的道或常俗之道。如李荣说：“道者，虚极之理也。夫论虚极之理，不可以有无分其象，不可以上下格其真。……圣人欲坦兹玄路，开以教门，借圆通之名，目虚极之理，以理可名，称之可道。故曰‘吾不知其名，字之曰道’。非常道者，非是人间常俗之道也。人间常俗之道，贵之以礼义，尚之以浮华，丧身以成名，忘己而徇利。”司马光说：“世俗之谈道者，皆曰道体微妙，不可名言。老子以为不然，曰道亦可言道耳，然非常人之所谓道也。……常人之所谓道者，凝滞于物。”裘锡圭说：“到目前为止，可以说，几乎从战国开始，大家都把‘可道’之‘道’……看成老子所否定的，把‘常道’‘常名’看成老子所肯定的。这种看法其实有它不合理的地方，……‘道’是可以说的。《老子》这个《道经》第一章，开宗明义是要讲他的‘道’。第一个‘道’字，理所应当，也是讲他要讲的‘道’：道是可以言说的。……那么这个‘恒’字应该怎么讲？我认为很简单，‘恒’字在古代作定语用，经常是‘平常’‘恒常’的意思。……‘道’是可以言说的，但是我要讲的这个‘道’，不是‘恒道’，它不是一般人所讲的‘道’。
'''
)
print(messages)

[SystemMessage(content='你将获得关于同一主题的2篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。', additional_kwargs={}, response_metadata={}), HumanMessage(content='1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。\n-----------\n2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。\n--------\n3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“

In [34]:
chat_result = chat_model.invoke(messages)
print(chat_result.content)

第一篇文章的论点是，“道可道”中的第一个“道”指的是道理，第二个“道”指言说的意思，第三个“道”指恒久存在的“道”，因此“道可道，非常道”意味着可以言说的道理不是恒久存在的道。第二篇文章的论点是，第一个“道”指宇宙万物的本原，第二个“道”指言说的意思，第三个“道”指恒久存在的道，因此，“道可道，非常道”表示可以言说的道并非恒久存在的道。

第二篇文章提出了更好的论点。这是因为它更直接地联系了“道”的本体和言说意义，解释了为什么可以言说的道不是恒久存在的道，这使得论点更加清晰明了。它还引用了多位学者对“道”概念的解释，呈现了更广泛的研究视角。另外，第二篇文章对“道”字的涵义解释更加一致和连贯，使读者更容易理解整个观点。


In [35]:
messages = summary_template.format_messages(
    num=2,
    user_input='''1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。
-----------
2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。
'''
)

In [36]:
chat_result = chat_model.invoke(messages)
print(chat_result.content)

第一个文章认为，“道可道”中的第一个“道”指的是道理，如仁义礼智之类；“可道”中的“道”指言说的意思；“常道”指恒久存在的“道”。因此，“道可道，非常道”指的是可以言说的道理，不是恒久存在的“道”。第二个文章认为，“道可道”中的第一个“道”指的是宇宙万物的本原；“可道”中的“道”指言说的意思；“常道”指恒久存在的“道”。因此，“道可道，非常道”指可以言说的“道”，就不是恒久存在的“道”。

第一个文章提出了更好的论点。它通过对“道可道”中每一个“道”的具体解释，以及引用了苏辙、蒋锡昌、陈鼓应等人的观点，来说明“道可道，非常道”所代表的含义。这种逐字解读以及引用权威观点的方式更有说服力，让读者更容易理解对“道”的不同解释。另一方面，第二个文章虽然也提供了观点，但对于“道”每一步骤的解释没有那么清晰，论证也相对不够充分。


### 使用 FewShotPromptTemplate 类生成 Few-shot Prompt 

构造 few-shot prompt 的方法通常有两种：
- 从示例集（set of examples）中手动选择；
- 通过示例选择器（Example Selector）自动选择.

In [37]:
from langchain.prompts.prompt import PromptTemplate


examples = [
  {
    "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里
"""
  },
  {
    "question": "craigslist的创始人是什么时候出生的？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日
"""
  },
  {
    "question": "乔治·华盛顿的外祖父是谁？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball
"""
  },
  {
    "question": "《大白鲨》和《皇家赌场》的导演是同一个国家的吗？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是
"""
  }
]

In [38]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)

# **examples[0] 是将examples[0] 字典的键值对（question-answer）解包并传递给format，作为函数参数
print(example_prompt.format(**examples[0]))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



In [39]:
print(example_prompt)

input_variables=['answer', 'question'] input_types={} partial_variables={} template='Question: {question}\n{answer}'


In [40]:
print(example_prompt.format(**examples[-1]))

Question: 《大白鲨》和《皇家赌场》的导演是同一个国家的吗？

这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是



#### 关于解包的示例

In [41]:
def print_info(question, answer):
    print(f"Question: {question}")
    print(f"Answer: {answer}")

print_info(**examples[0]) 

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？
Answer: 
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



### 生成 Few-shot Prompt

In [42]:
# 导入 FewShotPromptTemplate 类
from langchain.prompts.few_shot import FewShotPromptTemplate

# 创建一个 FewShotPromptTemplate 对象
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,           # 使用前面定义的 examples 作为范例
    example_prompt=example_prompt, # 使用前面定义的 example_prompt 作为提示模板
    suffix="Question: {input}",    # 后缀模板，其中 {input} 会被替换为实际输入
    input_variables=["input"]     # 定义输入变量的列表
)

# 使用给定的输入格式化 prompt，并打印结果
# 这里的 {input} 将被 "玛丽·波尔·华盛顿的父亲是谁?" 替换
print(few_shot_prompt.format(input="玛丽·波尔·华盛顿的父亲是谁?"))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里


Question: craigslist的创始人是什么时候出生的？

这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日


Question: 乔治·华盛顿的外祖父是谁？

这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball


Question: 《大白鲨》和《皇家赌场》的导演是同一个国家的吗？

这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是


Question: 玛丽·波尔·华盛顿的父亲是谁?


## 示例选择器 Example Selectors

**如果你有大量的参考示例，就得选择哪些要包含在提示中。最好还是根据某种条件或者规则来自动选择，Example Selector 是负责这个任务的类。**

BaseExampleSelector 定义如下：

```python
class BaseExampleSelector(ABC):
    """用于选择包含在提示中的示例的接口。"""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """根据输入选择要使用的示例。"""

```

`ABC` 是 Python 中的 `abc` 模块中的一个缩写，它表示 "Abstract Base Class"（抽象基类）。在 Python 中，抽象基类用于定义其他类必须遵循的基本接口或蓝图，但不能直接实例化。其主要目的是为了提供一种形式化的方式来定义和检查子类的接口。

使用抽象基类的几点关键信息：

1. **抽象方法**：在抽象基类中，你可以定义抽象方法，它没有实现（也就是说，它没有方法体）。任何继承该抽象基类的子类都必须提供这些抽象方法的实现。

2. **不能直接实例化**：你不能直接创建抽象基类的实例。试图这样做会引发错误。它们的主要目的是为了被继承，并在子类中实现其方法。

3. **强制子类实现**：如果子类没有实现所有的抽象方法，那么试图实例化该子类也会引发错误。这确保了继承抽象基类的所有子类都遵循了预定的接口。


In [43]:
# 导入需要的模块和类
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 定义一个提示模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],     # 输入变量的名字
    template="Input: {input}\nOutput: {output}",  # 实际的模板字符串
)

# 这是一个假设的任务示例列表，用于创建反义词
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

### Pandas 相关包首次导入错误后，再次执行即可正确导入

In [46]:

# 从给定的示例中创建一个语义相似性选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,                          # 可供选择的示例列表
    OpenAIEmbeddings(),                # 用于生成嵌入向量的嵌入类，用于衡量语义相似性
    Chroma,                            # 用于存储嵌入向量并进行相似性搜索的 VectorStore 类
    k=1                                # 要生成的示例数量
)

# 创建一个 FewShotPromptTemplate 对象
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 提供一个 ExampleSelector 替代示例
    example_prompt=example_prompt,      # 前面定义的提示模板
    prefix="Give the antonym of every input", # 前缀模板
    suffix="Input: {adjective}\nOutput:",     # 后缀模板
    input_variables=["adjective"],           # 输入变量的名字
)

In [47]:
# 输入是一种感受，所以应该选择 happy/sad 的示例。
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [48]:
# 输入是一种度量，所以应该选择 tall/short的示例。
print(similar_prompt.format(adjective="long"))

Give the antonym of every input

Input: tall
Output: short

Input: long
Output:


In [49]:
print(similar_prompt.format(adjective="rain"))

Give the antonym of every input

Input: windy
Output: calm

Input: rain
Output:
